In [186]:
#import random
import datetime
import time
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
#import jieba
#import jieba.analyse
#import gensim
#from sklearn.manifold import TSNE

from sklearn.cross_validation import train_test_split

%matplotlib notebook

plt.rcParams['font.sans-serif']=['SimHei'] # chinese

In [234]:
import apriori as ap
import fpgrowth as fp
import freq_patt_tree as fpt
from fastFM import als
from scipy import sparse

In [215]:
import imp
fp = imp.reload(fp)
ap = imp.reload(ap)

In [4]:
def build_fp_tree(file, support=2):
    data = fpg.loadData(file)
    tree, header = fpg.createTree(data, minSup=support)
    return tree, header

In [227]:
ap = imp.reload(ap)
time_s = time.time()
large_itemsets, recomm_rules = ap.run(ap.load_file('../data/sample_simple.csv'), 2, 0.8)
time_e = time.time()
print('time diff:', time_e - time_s)
ap.dump(large_itemsets, recomm_rules)

time diff: 0.0007319450378417969
Itemset: ('chicken',) support: 2.000
Itemset: ('mango',) support: 2.000
Itemset: ('rice', 'apple') support: 2.000
Itemset: ('rice', 'chicken') support: 2.000
Itemset: ('chicken', 'beer') support: 2.000
Itemset: ('milk', 'rice') support: 2.000
Itemset: ('milk', 'rice', 'beer') support: 2.000
Itemset: ('rice', 'chicken', 'beer') support: 2.000
Itemset: ('rice', 'apple', 'beer') support: 2.000
Itemset: ('milk', 'beer') support: 3.000
Itemset: ('apple', 'beer') support: 3.000
Itemset: ('apple',) support: 4.000
Itemset: ('milk',) support: 4.000
Itemset: ('rice',) support: 4.000
Itemset: ('rice', 'beer') support: 4.000
Itemset: ('beer',) support: 6.000
Rule: ('chicken',) => ('rice',) confidence: 1.000
Rule: ('chicken',) => ('beer',) confidence: 1.000
Rule: ('rice',) => ('beer',) confidence: 1.000
Rule: ('milk', 'rice') => ('beer',) confidence: 1.000
Rule: ('chicken',) => ('rice', 'beer') confidence: 1.000
Rule: ('rice', 'chicken') => ('beer',) confidence: 1.0

In [198]:
tree, header = build_fp_tree('../data/sample_simple.csv')
tree.disp(fname='out_fptree.txt')

-TreeRoot:1
--beer:6
---rice:2
----apple:2
-----chicken:1
---apple:1
---milk:3
----rice:2
-----chicken:1
--apple:1
---mango:1
--milk:1
---mango:1


In [244]:
fpt = imp.reload(fpt)
root, header = fpt.build_fp_tree(fpt.load_file('../data/sample_simple.csv'), min_sup=2)
root.dump()
header

-ROOT:1
--beer:6
---rice:2
----apple:2
-----chicken:1
---apple:1
---milk:3
----rice:2
-----chicken:1
--apple:1
---mango:1
--milk:1
---mango:1


{'apple': <freq_patt_tree.Node at 0x1a169ee160>,
 'beer': <freq_patt_tree.Node at 0x10e3f7780>,
 'chicken': <freq_patt_tree.Node at 0x1a169ee208>,
 'mango': <freq_patt_tree.Node at 0x10dc323c8>,
 'milk': <freq_patt_tree.Node at 0x10dc32208>,
 'rice': <freq_patt_tree.Node at 0x10e3f7828>}

In [221]:
fp.findPrefixPath('z', header['rice'][1])

{frozenset({'beer'}): 2, frozenset({'beer', 'milk'}): 2}

In [229]:
header['rice']

[4, <fpgrowth.treeNode at 0x108500588>]

In [ ]:
df = pd.read_csv('../data/sample_simple.csv', header=None)
df

In [8]:
def create_item_set_from_file(fn):
    itemset = set()
    fd = open(fn, 'r')
    for line in fd:
        for e in line.rstrip().split(','):
            itemset.add(e)
    fd.close()
    return itemset

In [9]:
def create_item_set(df):
    itemset = set()
    for index, row in df.iterrows():
        for e in row.tolist():
            itemset.add(e)
    itemset.remove(np.nan)
    return itemset

In [ ]:
itemset = create_item_set(df)
itemset

In [10]:
def create_onehot_vec(itemset):
    items = list(itemset)
    itemvec = sparse.eye(len(items))
    itemvec_dict = {}
    for i in range(len(items)):
        itemvec_dict[items[i]] = itemvec.getrow(i)
    return itemvec_dict

In [ ]:
itemvec = create_onehot_vec(itemset)
itemvec

In [ ]:
(itemvec["beer"] + itemvec["mango"]).todense()

In [ ]:
uservec = create_onehot_vec(df.index)

In [ ]:
uservec

In [6]:
def create_user_item_matrix(df, uservec, itemvec, mimic="SVD++"):
    mat = None
    for index, row in df.iterrows():
        vu = uservec[index]
        vi_sum = None
        if mimic != "MF":
            for item_name in row.tolist():
                if item_name is np.nan:
                    continue
                vi = itemvec[item_name]
                if vi_sum is None:
                    vi_sum = vi
                else:
                    vi_sum += vi
        if mimic == "SVD++":
            vi_sum /= np.sqrt(np.sum(vi_sum))
        for item_name in row.tolist():
            if item_name is np.nan:
                continue
            vi = itemvec[item_name]
            if mimic == "SVD++":
                vuil = sparse.hstack((vu, vi, vi_sum))
            elif mimic == "MF":
                vuil = sparse.hstack((vu, vi))
            if mat is None:
                mat = vuil
            else:
                mat = sparse.vstack((mat, vuil))
    return mat

In [7]:
def create_user_item_matrix_compacted_sid(df, uservec, itemvec, mimic="SVD++"):
    mat = None
    for index, row in df.iterrows():
        vu = uservec[index]
        vi_sum = None
        if mimic != "MF":
            for item_name in row['sid'].split(','):
                if item_name is np.nan:
                    continue
                vi = itemvec[item_name]
                if vi_sum is None:
                    vi_sum = vi
                else:
                    vi_sum += vi
        if mimic == "SVD++":
            vi_sum /= np.sqrt(np.sum(vi_sum))
        for item_name in row['sid'].split(','):
            if item_name is np.nan:
                continue
            vi = itemvec[item_name]
            if mimic == "SVD++":
                vuil = sparse.hstack((vu, vi, vi_sum))
            elif mimic == "MF":
                vuil = sparse.hstack((vu, vi))
            if mat is None:
                mat = vuil
            else:
                mat = sparse.vstack((mat, vuil))
    return mat

In [37]:
def create_user_item_predict_mat_compacted_sid(uid, df_sid, uservec, itemvec, mimic="SVD++"):
    mat = None
    vu = uservec[uid]
    if True:
        vi_sum = None
        if mimic != "MF":
            for item_name in df_sid.loc[uid]['sid'].split(','):
                if item_name is np.nan:
                    continue
                vi = itemvec[item_name]
                if vi_sum is None:
                    vi_sum = vi
                else:
                    vi_sum += vi
        if mimic == "SVD++":
            vi_sum /= np.sqrt(np.sum(vi_sum))
        for item_name in itemvec.keys():
            if item_name is np.nan:
                continue
            vi = itemvec[item_name]
            if mimic == "SVD++":
                vuil = sparse.hstack((vu, vi, vi_sum))
            elif mimic == "MF":
                vuil = sparse.hstack((vu, vi))
            if mat is None:
                mat = vuil
            else:
                mat = sparse.vstack((mat, vuil))
    return mat

In [ ]:
mat = create_user_item_matrix(df, uservec, itemvec, mimic="SVD++")
#mat = create_user_item_matrix(df, uservec, itemvec, mimic="MF")
#mat.todense()

In [ ]:
X = mat
y = np.ones(mat.shape[0])
X_ = sparse.csc_matrix(X)
X_train, X_test, y_train, y_test = train_test_split(X_, y)

In [ ]:
fm = als.FMRegression(n_iter=200, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)
y_pred = fm.predict(X_test)

In [ ]:
y_pred

In [ ]:
### apriori/fpgrowth

In [13]:
df_puffer = pd.read_excel('180419mark_satisfied_puffer.xlsx')

In [14]:
dfp = df_puffer.copy()

In [15]:
df1 = dfp[dfp['mark'] == 1].copy()

In [11]:
def print_apriori_rules(df_sidlookup, fn):
    def get_sid_info(df, sid):
        return df.loc[sid].iloc[0]['content'] + df.loc[sid].iloc[0]['slots']

    fd = open(fn, 'w')
    for index, row in df_rule.iterrows():
        for sid in row[1].split('|'):
            if sid == '':
                continue
            fd.write("<<< " + sid + ":" + get_sid_info(df_sidlookup, sid) + '\n')
        for sid in row[2].split('|'):
            if sid == '':
                continue
            fd.write(">>> " + sid + ":" + get_sid_info(df_sidlookup, sid) + '\n')
        fd.write('\n')
    fd.close()

In [47]:
def mark_sid_type(row):
    botid = row['source_type']
    sid = row['sid']
    if botid == "audio_music":
        row['sid'] = "m" + sid
    elif botid == "audio_unicast":
        row['sid'] = "u" + sid
    elif botid == "ai.dueros.bot.short_video":
        row['sid'] = "s" + sid
    elif botid == "ai.dueros.bot.video_on_demand":
        row['sid'] = "v" + sid
    return row

In [16]:
dfs = df1.apply(mark_sid_type, axis=1)

In [17]:
df_sidlookup = dfs[['sid', 'slots', 'content', 'nlu']].set_index('sid')
df_sidlookup.to_excel('out_sid_lookup.xlsx')
df_sidlookup.head()

,slots,content,nlu
sid,,,
m1138580000,"{""hmm_song"": ""温柔"", ""unit"": ""歌曲"", ""song"": ""温柔""}","{""content"": ""五月天，温柔"", ""type"": ""Text"", ""_ttsOpt...","{""domain"": ""audio.music"", ""intent"": ""audio.mus..."
m1110896524,"{""hmm_song"": ""凉凉"", ""unit"": ""歌曲"", ""song"": ""凉凉""}","{""content"": ""玄觞，凉凉"", ""type"": ""Text"", ""_ttsOpti...","{""domain"": ""audio.music"", ""intent"": ""audio.mus..."
s498542084515321457,"{""status"": ""0"", ""type"": ""相声"", ""keyword"": ""郭德纲""}","{""content"": """", ""type"": ""Text""}","{""domain"": ""short_video"", ""intent"": ""ai.dueros..."
u57729179189,"{""program_name"": ""波波放那克"", ""keyword_1"": ""那克"", ""...","{""content"": ""暂时没有此资源，为你推荐，五岁萌娃脱口秀%20duang~绝不加特...","{""domain"": ""audio.unicast"", ""intent"": ""audio.u..."
m964804248,"{""unit"": ""歌曲""}","{""content"": ""深夜时候适合听听独处的音乐，推荐给你，宋冬野，安和桥"", ""typ...","{""domain"": ""audio.music"", ""intent"": ""audio.mus..."


In [18]:
df1 = df_sidlookup.reset_index()

In [19]:
len(df1['sid'])

42180

In [20]:
df_sid = dfs[['uid', 'sid']].groupby('uid').agg(lambda x: ','.join(x))

In [ ]:
df_sid.to_csv('out_uid_sid.csv', index=False, header=None, sep='|')

In [111]:
tree, header = build_fp_tree('out_uid_sid.csv', support=10)

In [109]:
tree.disp(fname='out_fptree.txt', df=df_sidlookup)

-TreeRoot:1
--m991053430:47
---m1329025041:1
----m993189565:1
-----m1019301623:1
------v37d8667654cc4645b451236551db1141:1
-------m1322433906:1
--------m1223084284:1
---------m975025530:1
---m1365519490:1
----m1233414257:1
-----m984599676:1
------m1299328953:1
-------m1360527004:1
--------m1011151822:1
---------u57782223763:1
----------s15078821307677129444:1
---m1025707874:1
----m1029883736:1
-----m1046389703:1
------m1306305360:1
-------m1043344208:1
--------m1322433210:1
---------m1248969980:1
----------m995223533:1
-----------m1308142813:1
------------m1026707758:1
-------------m1288194089:1
---m1019301623:1
----m1013613347:1
---m1281941647:1
----s11302681162812298723:1
-----m1322431686:1
------m1117405477:1
-------m1090753072:1
---m1001219330:1
----m1241391273:1
-----s11302681162812298723:1
------m1360526302:1
-------m1051992469:1
--------m1004106362:1
---------m1327802859:1
----------m1360526749:1
-----------m1308270013:1
------------m1001172692:1
-------------m1343163929:1
---m1

------u56485315874:1
----m991053430:1
-----m1016326571:1
------m1019301623:1
-------m1005059134:1
----s6907243864958674208:1
-----m1001219330:1
------m998144258:1
-------m1322431569:1
--------m1039015538:1
---------m1016250500:1
----------m1002401720:1
-----------u85040421920:1
------------m973988277:1
----m1322431596:1
-----m1136066881:1
------m1052806803:1
-------m998663430:1
--------s9524856411733439290:1
---------m1322431566:1
----------m1234925521:1
-----------m993179695:1
------------m1100063384:1
-------------u57728623946:1
--------------m1317408992:1
---------------s8076579811795940842:1
---m1328615355:4
----m964804248:1
-----m1037242594:1
------m1025708042:1
-------u56486568164:1
--------m1010293394:1
---------m992112354:1
----m1093750375:1
-----s15447946718183289597:1
------m1037242594:1
-------m1034257022:1
--------m1004168019:1
---------m1027438381:1
----------m1043344208:1
-----------m1004615185:1
------------m1019619550:1
-------------m1308142813:1
--------------m99565568

-----m1374550706:1
------m1029883736:1
-------m997731459:1
--------m1009375043:1
---------u84986769595:1
----------u57729623413:1
-----------m1139176745:1
------------m1143892748:1
---m1233393653:4
----m1012282237:1
-----m1351597556:1
------m1322431596:1
-------m1322431614:1
--------m1299452734:1
---------m1005864835:1
----------m1022054770:1
-----------u56777508521:1
------------v58104838e510483f85074419c5d2f9ba:1
-------------s5116156294835343116:1
----s6907243864958674208:1
-----s12974410567250214554:1
------s11302681162812298723:1
-------s15447946718183289597:1
--------m1322431869:1
---------s6676773808377990636:1
----------v78e7e42b371567d9f3c58214c5feeda1:1
-----------s10993324362092403168:1
------------m1025707835:1
-------------s10094610749308229377:1
----m1008401098:1
-----m1093750375:1
------m997120535:1
-------m1224063886:1
--------m978045018:1
---------m997374012:1
----------s11673923141555725688:1
-----------m1005314929:1
------------vf45505041011499f8b4386a292e77435:1
---

----------m964759878:1
-----------m1014544123:1
------------m1322432583:1
-------------m1111539340:1
--------------s6143999550381754418:1
---------------m1004225346:1
----------------m991182322:1
-----------------s16704827125410062689:1
---m1322431614:1
----m1037242594:1
-----u57760297695:1
------u57779605418:1
---m1378473143:1
----m1039011785:1
-----m1307971155:1
------m1368905050:1
-------m1016280242:1
--------v3c0f1ee29171422fb1e7e880ebb04461:1
---m997120535:1
----m990590106:1
-----m1305302253:1
------m1013005708:1
-------m1088872831:1
--------m1004226551:1
---------m1125508961:1
----------m971155545:1
-----------m1322431680:1
------------m1116753052:1
-------------m1027213907:1
--------------m1004975420:1
---------------m989230656:1
----------------m1017418700:1
-----------------m1005962059:1
------------------m1004225346:1
---m980091060:1
---s11409760776512355215:1
----m1037242594:1
-----m1016039543:1
------u57782529583:1
-------u57388718416:1
--------m1009404635:1
---------u74475

-------------------------------s7678527194255808147:1
--------------------------------m994483228:1
---------------------------------s12765656016992815457:1
----------------------------------m1322431710:1
-----------------------------------s13674672424966518537:1
------------------------------------m1089643462:1
-------------------------------------m1322433522:1
--------------------------------------s7627171909580531808:1
---------------------------------------m1028831029:1
----------------------------------------m1322431692:1
-----------------------------------------s10137660742710162855:1
------------------------------------------s9103202547879641595:1
-------------------------------------------m1360526014:1
--------------------------------------------m1327802730:1
---------------------------------------------m1370492522:1
----------------------------------------------m1360526089:1
-----------------------------------------------m1140120763:1
-------------------------------------------

...

-------m1013116079:1
--------m1019926213:1
---------m979770756:1
---m1327881243:2
----m1001165755:1
-----m1351721093:1
------m997731459:1
-------m1133954773:1
--------m1296231150:1
----m1017537898:1
-----m996673889:1
------m1322432367:1
---v6994bfa984394451955cb7d518def182:1
----m1141773023:1
---m1039011785:1
----m1092511753:1
-----m1376748221:1
---m1029111877:1
----m1003309624:1
-----m1105737158:1
------m1041881756:1
-------m996469598:1
---m1290719507:1
----m1360501102:1
---m1312164601:1
----m1322432736:1
--m1001219330:37
---m998144258:2
----v679cb853037f453cb83a4599e3072dd2:1
-----m1032230936:1
------m997380165:1
-------m1025707874:1
--------m1019879977:1
---------m1322433222:1
----m1032856732:1
-----v679cb853037f453cb83a4599e3072dd2:1
------m998663430:1
-------u58309976738:1
--------m1375922663:1
---------m1005314929:1
----------m1009061423:1
-----------m1257842790:1
------------m988942596:1
-------------m996292539:1
--------------u57728623946:1
---------------m1119434275:1
--------

--------------m990410542:1
---------------m1322431677:1
----------------m1003309624:1
-----------------m992112354:1
---s11725220996556825632:1
----s912338179177576920:1
-----m1003246898:1
------m1322432025:1
-------s8852516731159762172:1
---m1004725438:2
----m1360526302:1
-----m1051992469:1
------m1360527319:1
-------m990459927:1
--------m1365216262:1
---------u57803345634:1
----------m1003712075:1
-----------m1125202424:1
------------m1370494604:1
-------------m1004497072:1
--------------m1112294080:1
---------------m1322433303:1
----------------m1019092547:1
-----------------m1360525951:1
------------------m1042010969:1
-------------------m1054931184:1
--------------------m1008144319:1
----m1301765682:1
-----m997380165:1
------m1005435024:1
-------m1356775019:1
--------m1327707990:1
---------m1031490812:1
----------m1004677913:1
-----------m1252907708:1
---s8338640772925345646:2
----u57739841667:1
-----m1377364526:1
------s11302681162812298723:1
-------m1001165755:1
--------m10041063

---------------m1367882044:1
----------------u57782658148:1
-----------------u57760605834:1
------------------m1264758453:1
-------------------m1017837173:1
--------------------m1256793394:1
---------------------u72864455847:1
----------------------m1367501416:1
-----------------------m1034166356:1
------------------------m976781478:1
---v679cb853037f453cb83a4599e3072dd2:1
----m994482868:1
-----vd9d6363fdb284796b160fc55d092a77f:1
------vb2d6200694e1c59e15b449c60c9626e2:1
-------m1093289710:1
--------m1322431584:1
---------u57779605418:1
----------u57388718416:1
-----------m1360525978:1
------------m1322431647:1
-------------m1075450977:1
--------------m1008206774:1
---------------m1322432259:1
----------------m1370492738:1
---m1037043782:2
----m1283527062:1
-----m1322432181:1
------m1290725523:1
-------m964826082:1
--------m1103315822:1
---------m1010100227:1
----------m998070878:1
-----------vc200174fceeb8916af6c08ccc8654452:1
------------m997840948:1
----m1327802934:1
---m1233414257:

---------viqiyi_mobile_180778201:1
----------m1322432169:1
---m978870771:1
----m1168883514:1
-----m998606152:1
--m1026810877:91
---s794585456984438506:1
----m1092128721:1
-----u72660370122:1
------s14034722543172334331:1
---m1029095653:4
----m964466133:1
-----v17be0f76272feec3c7d92512125cf4c6:1
------m1052156413:1
-------m1112410753:1
--------m1017551038:1
---------m1365323785:1
----------m1322431608:1
-----------m1303662297:1
------------m1019741800:1
-------------m1009424927:1
--------------m1105737158:1
---------------m1323079872:1
----------------m1005448766:1
-----------------m1096898970:1
------------------m1054931184:1
----m1322431596:1
-----m1100452127:1
------m987285300:1
-------m1368855664:1
--------m1360526188:1
---------m1368905050:1
----------m997374012:1
-----------m1037043782:1
------------m1353484817:1
-------------m1312164601:1
--------------m1002537134:1
---------------m1013186908:1
----------------m1125508961:1
-----------------m1327484919:1
------------------m130811

KeyboardInterrupt: 

In [ ]:
large_itemsets, recomm_rules = run_apriori('out_uid_sid.csv', 0.001, 0.2)
apriori.printResults(large_itemsets, recomm_rules, fn_items='out_large_itemsets.csv', fn_rules='out_recomm_rules.csv')

In [194]:
ap = imp.reload(ap)
time_s = time.time()
large_itemsets, recomm_rules = ap.run(ap.load_file('out_uid_sid.csv'), 0.001, 0.2, output_support_only=False)
time_e = time.time()
print('time diff:', time_e - time_s)

time diff: 332.24187088012695


In [195]:
ap = imp.reload(ap)
time_s = time.time()
large_itemsets, recomm_rules = ap.run(ap.load_file('out_uid_sid.csv'), 0.001, 0.2, output_support_only=True)
time_e = time.time()
print('time diff:', time_e - time_s)

time diff: 332.4704828262329


In [184]:
ap.dump(large_itemsets, recomm_rules)

Itemset: ('m1094834137',) support: 0.010
Itemset: ('m1013005708',) support: 0.010
Itemset: ('m1345521604',) support: 0.010
Itemset: ('m1012283935',) support: 0.010
Itemset: ('m1374550706',) support: 0.010
Itemset: ('m1055706684',) support: 0.011
Itemset: ('m1002987961',) support: 0.011
Itemset: ('m1112410753',) support: 0.011
Itemset: ('s11409760776512355215',) support: 0.011
Itemset: ('m1037242594',) support: 0.011
Itemset: ('m1009362125',) support: 0.011
Itemset: ('m997374012',) support: 0.011
Itemset: ('m1290719507',) support: 0.011
Itemset: ('u58356824385',) support: 0.011
Itemset: ('m1375922663',) support: 0.011
Itemset: ('m990759337',) support: 0.011
Itemset: ('s5015996802295282022',) support: 0.011
Itemset: ('m978045018',) support: 0.011
Itemset: ('m1023707444',) support: 0.011
Itemset: ('v6994bfa984394451955cb7d518def182',) support: 0.011
Itemset: ('u58309976738',) support: 0.011
Itemset: ('m1367287708',) support: 0.011
Itemset: ('m1356775019',) support: 0.011
Itemset: ('m10162

In [ ]:
dfs[dfs['sid'] == 'm1023606622'].head()[['resource', 'query']]

In [ ]:
df_rule = pd.read_csv('out_recomm_rules_20%.csv', header=None)

In [ ]:
print_apriori_rules(df_sidlookup, 'out_rules_display.txt')

In [ ]:
### SVD++

In [21]:
df_sid.head()

,sid
uid,
3F180611863094C1,"s8563948906666203907,s15703777828732809183,u57..."
3F180611863465A0,"m993189565,m1019301623,m1131777838,m1017824831..."
3F180611863C5F1A,"m1012282237,s11584181565836183544,m1093750375,..."
3F180611863E1288,"m1138580000,m1110896524,s498542084515321457,u5..."
3F180611864E141B,"s3019915722752617984,s3019915722752617984,m101..."


In [22]:
itemset = create_item_set_from_file('out_uid_sid.csv')

In [23]:
itemvec = create_onehot_vec(itemset)

In [24]:
len(itemvec)

14429

In [25]:
itemvec

{'s15232494086522288041': <1x14429 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 's18198521607573341674': <1x14429 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'u57716089739': <1x14429 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'm995170077': <1x14429 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 's17248721292431073123': <1x14429 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 's4820830559391402438': <1x14429 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'u56509278399': <1x14429 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'm1037927150': <1x14429 sparse matr

In [26]:
uservec = create_onehot_vec(df_sid.index)

In [27]:
uservec

{'3F180611863094C1': <1x3113 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 '3F180611863465A0': <1x3113 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 '3F180611863C5F1A': <1x3113 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 '3F180611863E1288': <1x3113 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 '3F180611864E141B': <1x3113 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 '3F18061186542DB1': <1x3113 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 '3F1806118656072B': <1x3113 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 '3F180611865841F3': <1x3113 sparse matrix of ty

In [28]:
df_sid.loc['3F18061186542DB1']

sid    m1110896524,m1322431569,m1028177873,s501599680...
Name: 3F18061186542DB1, dtype: object

In [29]:
mat = create_user_item_matrix_compacted_sid(df_sid, uservec, itemvec, mimic="SVD++")

In [ ]:
itemvec['u57751445013']

In [30]:
X = mat
y = np.ones(mat.shape[0])
X_ = sparse.csc_matrix(X)
X_train, X_test, y_train, y_test = train_test_split(X_, y)

In [41]:
fm = als.FMRegression(n_iter=200, init_stdev=0.1, rank=50, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)
y_pred = fm.predict(X_test)

In [32]:
y_pred

array([0.99991711, 0.99934209, 0.99962985, ..., 1.00008228, 0.99996201,
       0.99999391])

In [38]:
X_eval = create_user_item_predict_mat_compacted_sid('3F180611863094C1', df_sid, uservec, itemvec, mimic="SVD++")

In [42]:
y_eval = fm.predict(X_eval)

In [43]:
y_eval[:200]

array([0.99988672, 0.99996435, 0.9992056 , 0.99966375, 1.00001566,
       0.99966545, 0.99981872, 0.99965643, 1.00001566, 0.99948935,
       0.99958038, 0.99924043, 1.00001566, 0.99964825, 1.00001566,
       0.99996826, 0.99960466, 0.99938487, 0.99959003, 0.99942239,
       0.99992157, 0.99932322, 0.99956851, 0.9994798 , 0.99992652,
       0.99955839, 1.00001566, 0.9994239 , 0.99970987, 0.99954138,
       1.00001566, 1.00001566, 0.99975058, 0.99990231, 0.99955402,
       0.9995937 , 1.00001566, 0.99949254, 0.99866835, 0.99982106,
       0.99946327, 0.99992137, 0.99954837, 0.99955839, 1.00001566,
       1.00056117, 0.9993524 , 0.99955467, 0.9993766 , 0.9993876 ,
       0.99999495, 0.99971542, 0.99916557, 0.99960154, 0.99960939,
       1.00001566, 0.99948893, 0.99929827, 1.00001566, 0.99958986,
       1.00001566, 0.99945611, 0.99967142, 0.99993787, 0.9996865 ,
       0.99966972, 0.99977146, 0.9994482 , 0.99950452, 0.99889048,
       0.99944828, 0.99961169, 0.99996136, 0.99947821, 0.99945

In [ ]:
###